# ファインチューニングの実装
- GPUの使用
- ファインチューニング
- パラメータの保存/読み出し

## AWSクラウドGPUの使用方法
強いEC2立ててそこで動かすだけなのでここに追記  
  
GPUマシンとしては`p2.xlarge`を用いる．  
一番安価な強力GPUマシン  
料金は約100円/hour(米国サーバなら)  
メモリサイズは約64GB  
ストレージは75GBだが，必要に応じて200GBなどに増やす  
あとはサーバー立てたらSSH接続して環境整えてリポジトリもらって実行

## ファインチューニング
転移学習では一部のみに対して再学習させたり，モデルへの変更を加えたりした．  
対してファインチューニングは，全ての層のパラメータを再学習させる  
入力層に近い所は学習率を小さく，出力層に近い所は大きくするのが一般的

## フォルダ準備と事前準備
1-1のフォルダ準備を行う  
また，GPUマシン上で学習を実行する．

## DatasetとDataLoaderを作成
1-3の
- ImageTransform
- make_datapath_list
- HymenpopteraDataset

を再利用

In [1]:
import glob
import os
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

In [2]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [3]:
data_dir = "../../datasets/ptca_datasets/chapter1"
hymenoptera_path = os.path.join(data_dir, "hymenoptera_data")

In [4]:
class ImageTransform():
    """
    画像の前処理＋DataAug
    
    Attributes
    ----------
    resize: int
        リサイズ先の大きさ
    mean: (R, G, B)
        各色チャネルの平均値
    std: (R, G, B)
        各色チャネルの標準偏差
    """
    
    def __init__(self, resize, mean, std):
        self.data_transform = {
            "train": transforms.Compose([
                transforms.RandomResizedCrop(
                    resize, scale=(0.5, 1.0)
                ), # DataAug
                transforms.RandomHorizontalFlip(), # DataAug
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            "val": transforms.Compose([
                transforms.Resize(resize),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }
        
    def __call__(self, img, phase="train"):
        """
        Parameters
        ----------
        phase: 'train' or 'val'
            前処理のモードを指定
        """
        return self.data_transform[phase](img)
        

In [5]:
def make_datapath_list(phase="train"):
    """
    データのパスを格納したリストを作成する
    
    Parameters
    ----------
    phase: 'train' or 'val'
        訓練データか検証データかを指定する
    
    Returns
    -------
    path_list: list
        データへのパスを格納したリスト
    """
    
    rootpath = hymenoptera_path
    target_path = osp.join(f"{rootpath}/{phase}/**/*.jpg")
    print(target_path)
    
    path_list = []
    for path in glob.glob(target_path):
        path_list.append(path)
    
    return path_list

In [6]:
class HymenopteraDataset(data.Dataset): # PytorchのDatasetクラスを継承
    """
    アリとハチの画像のDatasetクラス
    
    Attributes
    ----------
    file_list: リスト
        画像のパスを格納したリスト
    transform: object
        前処理クラスのインスタンス
    phase: 'train' or 'test'
        学習データかテストデータかを指定する
    """
    
    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        
        img_transformed = self.transform(img, self.phase)
        
        label = img_path.split(os.sep)[-2]
        
        if label == "ants":
            label = 0
        elif label == "bees":
            label = 1
        
        return img_transformed, label

In [7]:
batch_size = 32
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
transform = ImageTransform(size, mean, std)

train_list = make_datapath_list(phase="train")
train_dataset = HymenopteraDataset(train_list, transform, "train")
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)

val_list = make_datapath_list(phase="val")
val_dataset = HymenopteraDataset(val_list, transform, "val")
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False
)

dataloaders_dict = {
    "train": train_dataloader,
    "val": val_dataloader
}

../datasets/chapter1/hymenoptera_data/train/**/*.jpg
../datasets/chapter1/hymenoptera_data/val/**/*.jpg


## ネットワークモデルを作成

これも1-3と同じ

In [8]:
# VGG-16の学習済みモデルを使用
net = models.vgg16(pretrained=True)

# 2択の全結合層を加える
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# 訓練モード
net.train()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## 損失関数を定義
これも1-3と同じ

In [9]:
criterion = nn.CrossEntropyLoss()

## 最適化手法を設定
転移学習とは異なり，全ての学習可能パラメータを再学習できるよう設定する  

In [10]:
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

update_param_names_1 = [
    "features"
]
update_param_names_2 = [
    "classifier.0.weight",
    "classifier.0.bias",
    "classifier.3.weight",
    "classifier.3.bias"
]
update_param_names_3 = [
    "classifier.6.weight",
    "classifier.6.bias"
]

for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1に格納: ", name)
    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2に格納: ", name)
    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3に格納: ", name)
    else:
        param.requires_grad = False
        print("勾配計算なし， 学習しない: ", name)

params_to_update_1に格納:  features.0.weight
params_to_update_1に格納:  features.0.bias
params_to_update_1に格納:  features.2.weight
params_to_update_1に格納:  features.2.bias
params_to_update_1に格納:  features.5.weight
params_to_update_1に格納:  features.5.bias
params_to_update_1に格納:  features.7.weight
params_to_update_1に格納:  features.7.bias
params_to_update_1に格納:  features.10.weight
params_to_update_1に格納:  features.10.bias
params_to_update_1に格納:  features.12.weight
params_to_update_1に格納:  features.12.bias
params_to_update_1に格納:  features.14.weight
params_to_update_1に格納:  features.14.bias
params_to_update_1に格納:  features.17.weight
params_to_update_1に格納:  features.17.bias
params_to_update_1に格納:  features.19.weight
params_to_update_1に格納:  features.19.bias
params_to_update_1に格納:  features.21.weight
params_to_update_1に格納:  features.21.bias
params_to_update_1に格納:  features.24.weight
params_to_update_1に格納:  features.24.bias
params_to_update_1に格納:  features.26.weight
params_to_update_1に格納:  features.26.bias


最適化手法には，今回もMomentum SGDを用いる．  
パラメータごとに最適化手法のパラメータを設定できる．  
momentumは全て同じにするので外側に書いているが，個別に設定することも可能な模様

In [11]:
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3},
], momentum=0.9)

## 学習・検証を実施
基本的に1-3と同じだが，GPUを使用できるようにしているところが異なる  

In [12]:
def set_device(net, benchmark=True):
    # GPUが使用可能ならば設定
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス: ", device)
    
    # ネットワークをGPUへ渡す
    net.to(device)
    
    # ネットワークがある程度固定であれば，高速化できる
    torch.backends.cudnn.benchmark = benchmark
    
    return device

In [13]:
# 1-3とほぼ同様の関数だが，データをGPUへ送る必要がある
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs, device=None):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("--------------------")
        
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()
            
            epoch_loss = 0.0
            epoch_corrects = 0
            epoch_size = len(dataloaders_dict[phase].dataset)
            
            # 未学習時の検証性能を確かめるため，最初の訓練は省略
            if (epoch == 0) and (phase == 'train'):
                continue
            
            # ミニバッチごとに学習を行う
            # tqdm: means "progress" in Arabic (taqadum)
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                
                # 使用可能デバイスがあるならデータを渡す
                if device:
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                
                    # 訓練時は誤差逆伝播で勾配を取得して重みの更新を行う
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                    # ミニバッチのLossと正解数をepochのものに追加
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)
            
            # item一つごとの平均Lossと平均正解数
            epoch_loss = epoch_loss / epoch_size
            epoch_acc = epoch_corrects.double() / epoch_size
            
            # Epochのステータス
            print(f'{phase} Loss: {epoch_loss} Acc: {epoch_acc}')

In [14]:
num_epochs = 2
device = set_device(net)
train_model(
    net=net, 
    dataloaders_dict=dataloaders_dict, 
    criterion=criterion, 
    optimizer=optimizer, 
    num_epochs=num_epochs, 
    device=device
)

使用デバイス:  cuda:0


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/2
--------------------


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 0.7703910294701072 Acc: 0.4444444444444445
Epoch 2/2
--------------------


  0%|          | 0/5 [00:00<?, ?it/s]

train Loss: 0.4966980400399416 Acc: 0.7489711934156379


100%|██████████| 5/5 [00:01<00:00,  4.19it/s]

val Loss: 0.17173519048815458 Acc: 0.9477124183006537


## 学習したネットワークを保存・ロード

In [15]:
net_path = osp.join(data_dir, "weights_fine_tuning.pth")

In [16]:
# セーブ, state_dictの返り値を保存
torch.save(net.state_dict(), net_path)

In [17]:
# ロード, GPUへ重みをあらかじめ渡すことができる
if not torch.cuda.is_available():
    weights = torch.load(net_path)
else:
    weights = torch.load(net_path, map_location={'cuda:0': 'cpu'})

net.load_state_dict(weights)

<All keys matched successfully>